title: MCU categories
author: Vladas Jankus 
date: 2022-03-31
region: EU  
tags: cards, card, MCU, product, lapse, MCU period, MCU rate, MCU lifetime rate 
summary: The reasearch looks at categorising Monthly Cards Users (MCU). Provides basic insights into the new metric, looks at MCU conversion, compares MCU against MAU.

In [1]:
cd /app/

/app


In [2]:
! pip install altair
import altair as alt
import pandas as pd
from utils.datalib_database import df_from_sql
from IPython.display import HTML, Markdown as md

     |████████████████████████████████| 812 kB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 55 kB 1.4 MB/s eta 0:00:01
You should consider upgrading via the '/usr/local/bin/python -m pip install --upgrade pip' command.


In [3]:
def style_theme():
    # Typography
    font = "Karla"
    # At Urban it's the same font for all text but it's good to keep them separate in case you want to change one later.
    labelFont = "Montserrat"
    sourceFont = "Montserrat"
    # Axes
    axisColor = "#000000"
    gridColor = "#DEDDDD"
    # Colors
    markColor = "#88C04D"
    main_palette = [
        "#88C04D",
        "#44A8DA",
        "#FED234",
        "#A989C5",
        "#066696",
        "#FC97C2",
        "#FF8360",
        "#B7ABC2",
    ]
    sequential_palette = [
        "#cfe8f3",
        "#a2d4ec",
        "#73bfe2",
        "#46abdb",
        "#1696d2",
        "#12719e",
    ]
    return {
        # width and height are configured outside the config dict because they are Chart configurations/properties not chart-elements' configurations/properties.
        "width": 700,  # from the guide
        "height": 380,  # not in the guide
        "config": {
            "title": {
                "fontSize": 18,
                "font": font,
                "anchor": "start",  # equivalent of left-aligned.
                "fontColor": "#000000",
            },
            "axisX": {
                "domain": True,
                "domainColor": axisColor,
                "domainWidth": 1,
                "grid": False,
                "labelFont": labelFont,
                "labelFontSize": 12,
                "labelAngle": 0,
                "tickColor": axisColor,
                "tickSize": 5,  # default, including it just to show you can change it
                "titleFont": font,
                "titleFontSize": 12,
                "titlePadding": 10,  # guessing, not specified in styleguide
                "title": "X Axis Title (units)",
            },
            "axisY": {
                "domain": False,
                "grid": True,
                "gridColor": gridColor,
                "gridWidth": 1,
                "labelFont": labelFont,
                "labelFontSize": 12,
                "labelAngle": 0,
                "ticks": False,  # even if you don't have a "domain" you need to turn these off.
                "titleFont": font,
                "titleFontSize": 12,
                "titlePadding": 10,  # guessing, not specified in styleguide
                "title": "Y Axis Title (units)",
                # titles are by default vertical left of axis so we need to hack this
                "titleAngle": 0,  # horizontal
                "titleY": -15,  # move it up
                "titleX": 18,  # move it to the right so it aligns with the labels
            },
            "range": {
                "category": main_palette,
                "diverging": sequential_palette,
            },
            "legend": {
                "labelFont": labelFont,
                "labelFontSize": 12,
                "symbolType": "square",  # just 'cause
                "symbolSize": 100,  # default
                "titleFont": font,
                "titleFontSize": 12,
                "title": "",
                "orient": "none",
                "direction": "horizontal",
                "titleAnchor": "middle",
            },
            "view": {
                "stroke": "transparent",  # altair uses gridlines to box the area where the data is visualized. This takes that off.
            },
            "background": {"color": "#D9E9F0"},  # white rather than transparent
            ### MARKS CONFIGURATIONS ###
            "area": {
                "fill": markColor,
            },
            "line": {
                "color": markColor,
                "stroke": markColor,
                "strokeWidth": 5,
            },
            "trail": {
                "color": markColor,
                "stroke": markColor,
                "strokeWidth": 0,
                "size": 1,
            },
            "path": {
                "stroke": markColor,
                "strokeWidth": 0.5,
            },
            "point": {
                "filled": True,
            },
            "text": {
                "font": sourceFont,
                "color": markColor,
                "fontSize": 11,
                "align": "right",
                "fontWeight": 400,
                "size": 11,
            },
            "bar": {
                "size": 40,
                "binSpacing": 1,
                "opacity": 0.8,
                "continuousBandSize": 30,
                "discreteBandSize": 30,
                "fill": markColor,
                "stroke": False,
            },
        },
    }


alt.themes.register("style_theme", style_theme)
alt.themes.enable("style_theme")

ThemeRegistry.enable('style_theme')

In [4]:
sql_path = "research/product/cards/20220220_mcu_categories/queries/"

In [5]:
query = open(sql_path + "mcu_periods.sql", "r").read()
mcu_periods = df_from_sql("redshiftreader", query)

In [6]:
query = open(sql_path + "mcu_rate.sql", "r").read()
mcu_rate = df_from_sql("redshiftreader", query)

In [7]:
query = open(sql_path + "mcu_rate_vs_periods.sql", "r").read()
mcu_rate_vs_periods = df_from_sql("redshiftreader", query)

In [8]:
query = open(sql_path + "mcu_rate_vs_lapse.sql", "r").read()
mcu_rate_vs_lapse = df_from_sql("redshiftreader", query)

In [9]:
query = open(sql_path + "mcu_categories.sql", "r").read()
mcu_categories = df_from_sql("redshiftreader", query)

# MCU categorisation
Vladas Jankus<br/>
2022-03-31

This short study will look at the ways MCU customers can be categorised into different groups according to the common traits that they display in their card activity. This study can be used for reference in further research when MCU categorisation is required.

## TL;DR
We can currently distinguish three categories of active MCUs and two categories of inactive MCUs. In total five MCU categories + potential MCUs. Below are all MCU groups listed, please read the study to understand how we came up with them.
1. **Continuous MCUs** - all active MCUs with over 90% of MCU activity. We can expect these customers to be the drivers of all KPIs. An assumption that can later be tested and hopefully confirmed.
2. **Sporadic MCUs** - all active MCUs that have already lapsed 3+ times. Showing that they tend to lapse and return using their card.
3. **Undecided MCUs** - these are all active MCUs that are <90% with their overall activity and have 1-2 MCU periods. Showing that they don't lapse frequently. These MCUs should either end up with Sporadic MCUs if they stop and return a couple of more times, or end up in the continuous MCU bracket if they remain continuously active.
4. **Dormant MCUs** - all inactive MCUs that have been inactive for less than 6 months. They can still potentially return using their card and could show similar traits to sporadic MCUs.
5. **Lapsed MCUs** - all inactive MCUs that have been inactive for over 6 months. They are unlikely to return using their card.
6. **Potential MCUs** - these are the customers that have topped up and have not used their card yet, but have an open N26 account. They are not really MCUs, but for some analyses it might be useful to include this group. Let's see how this split looks on our current MCU base.

Please feel free to email or slack the author if you see errors or have suggestions!

### Contents:
* [1. Count of MCU periods](#section1)
* [2. MCU rate](#section2)
* [3. MCU rate cross with MCU periods](#section3)
* [4. MCU categorisation by lapse](#section4)
* [5. A word on MCU periods vs. activity vs. lapse](#section5)
* [6. Final categorisation](#section6)

#### Terms
Here you can find main terms that are going to be used here.
1. MCU - monthly card user.
2. MCU period - a stretch of time that customer has been an MCU. After card use a customer becomes an MCU for 35 days and this period is extended with continuous card usage. One customer can have multiple MCU periods e.g. when they stop using their card for long (>35 days) stretches and then return.
3. MCU rate - percentage of time the customer was an MCU. e.g. 90% means that the customer was active with their card 90% of the time.
4. Dormant vs lapsed MCU - we distinguish two types of inactive MCUs based on the likelihood of returning. Dormant MCU is someone who lapsed and will likely return using their card. Lapsed MCU is a customer who has an open N26 account, but is unlikely to return using his card.
5. ftMAU - customer who has topped up his account and therefore has been a first time MAU.

## 1. Count of lapses <a name="section1"></a>
The first obvious category to explore is the count of lapses. This causes customers to have multiple MCU periods. The main assumption here is, that a customer with one long MCU period displays a very different behaviour from a customer that keeps lapsing and relapsing.

Note that the chart below shows MCU periods. Every MCU period ends with a lapse. customers with 0 MCU periods are the ones who have an open N26 account, have topped up, but have never made a card transaction.

In [10]:
data = mcu_periods
xaxis = "mcu_periods"
yaxis = "rate"

x_axis_title = "Separate MCU periods"
y_axis_title = "Percentage of customers"
chart_title = "ftMAU division by count of their separate MCU periods"

base = alt.Chart(data).encode(x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title)))

bar = base.mark_bar(size=100).encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(),
    ),
    color=alt.Color("mcu_periods", legend=None),
)
bar_labels = bar.mark_text(dx=40, dy=25, size=40).encode(
    y=alt.Y(yaxis + ":Q", axis=None), text=alt.Text(yaxis + ":N", format=".0%")
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O"),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
text_bar = base.mark_text(align="left", dx=-33, dy=25, size=40, color="#fff").encode(
    y=alt.Y(yaxis + ":Q", axis=None),
    text=alt.condition(nearest, yaxis + ":Q", alt.value(" "), format=".0%"),
)
bar_select = bar.mark_bar(size=100).encode(
    y=alt.Y(yaxis + ":Q", axis=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)

alt.layer(bar, bar_labels, selectors, bar_select, text_bar).properties(
    title=chart_title
).resolve_scale(y="independent").configure(background="#fff")

alt.LayerChart(...)

There are several issues with this kind of split. If we look at the column with customers who had only 1 MCU period, possibly several very different types of customers are hiding here. One type of customer is a long-term continuous MCU that started 2 years ago and has never stopped using his card, another one is a customer that made one transaction and stopped. 

Furthermore, customers in other columns can also be very different. We know that the ones in the 5+ column keep lapsing and returning to using their card, but we don't really know how much time they spent using their card and how many transactions they can generate.

## 2. MCU rate <a name="section2"></a>

MCU rate can show us what distinct MCU periods cannot. In the end we don't care whether the customer has used our card continuously or not, as long as he keeps a high percentage of use. I have already explained the concept of MCU rate with a [customer example](https://research.tech26.de/reports/20211206_mcu_insights.html#12) in an earlier study.

We can now look how our customer base is divided based on the percentage of time spent with an MCU status.

In [11]:
data = mcu_rate
xaxis = "activity_level"
yaxis = "cst_rate"

x_axis_title = "MCU activity level"
y_axis_title = "Percentage of customers"
chart_title = "ftMAUs split by their MCU rate into activity levels"

column_order = [
    "None (0%)",
    "Low (1%-20%)",
    "Sporadic (20%-40%)",
    "Mid (40%-70%)",
    "High (70%-90%)",
    "Continuous (90%-100%)",
]

base = alt.Chart(data).encode(
    x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title), sort=column_order)
)

bar = base.mark_bar(size=100).encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(),
    ),
    color=alt.Color(xaxis, legend=None),
)
bar_labels = bar.mark_text(dx=40, dy=25, size=40).encode(
    y=alt.Y(yaxis + ":Q", axis=None), text=alt.Text(yaxis + ":N", format=".0%")
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O", sort=column_order),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
text_bar = base.mark_text(align="left", dx=-33, dy=25, size=40, color="#fff").encode(
    y=alt.Y(yaxis + ":Q", axis=None),
    text=alt.condition(nearest, yaxis + ":Q", alt.value(" "), format=".0%"),
)
bar_select = bar.mark_bar(size=100).encode(
    y=alt.Y(yaxis + ":Q", axis=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)

alt.layer(bar, bar_labels, selectors, bar_select, text_bar).properties(
    title=chart_title
).resolve_scale(y="independent").configure(background="#fff")

alt.LayerChart(...)

From this chart we can see that around 28% of our eligible customers use their card continuously for over 90% of their lifetime. The others are distributed somewhat equally among the rest of brackets. 

Customers with _None (0%)_ activity are the ones who have topped up their account but have never started using their card (and haven't closed the account either). You can notice the same 11% as in the chart above. Sometimes we will choose to ignore this type of customer because they have no transactional data.

# 3. MCU rate cross with MCU periods<a name="section3"></a>
The MCU rate already tells us a lot more about the card users. We can cross the two charts above to see how often the customers in each MCU rate bracket relapse. Mostly we want to see whether there are behavioural differences that could allow us to assume each bracket holds a different type of a customer.

In [12]:
data = mcu_rate_vs_periods
xaxis = "activity_level"
yaxis = "customer_percentage"
col_split = "mcu_periods"

x_axis_title = "MCU activity level"
y_axis_title = "Percentage of customers"
chart_title = "MCUs split by their MCU rate into activity levels"

column_order = [
    "None (0%)",
    "Low (1%-20%)",
    "Sporadic (20%-40%)",
    "Mid (40%-70%)",
    "High (70%-90%)",
    "Continuous (90%-100%)",
]

base = alt.Chart(data).encode(
    x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title), sort=column_order)
)

bar = base.mark_bar(size=100).encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(),
        stack="normalize",
    ),
    color=alt.Color(
        col_split, legend=alt.Legend(title="MCU periods", legendX=250, legendY=-55)
    ),
)
bar_inside_labels = base.mark_text(size=20, dx=0, dy=-8).encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(),
        stack="normalize",
    ),
    color=alt.Color(col_split, legend=None),
    text=alt.Text("bracket_rate:Q", format=".0%"),
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O", sort=column_order),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)

bar_select = bar.mark_bar(size=100).encode(
    y=alt.Y(yaxis + ":Q", axis=None, scale=alt.Scale(), stack="normalize"),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)

alt.layer(
    bar + bar_inside_labels,
    bar_select + bar_inside_labels,
    selectors
    # text_bar
).properties(title=chart_title).resolve_scale(y="independent").configure(
    background="#fff"
)

alt.LayerChart(...)

As we go more to between 20% and 90% of MCUs there are significantly more customers with 3+ activity periods. We could assume this shows a different type of customer. Here we can outline several main MCU types:

1. No MCU activity - 0%: These are the customers that have topped up the account but not made a card transaction. Obviously all of them have 0 MCU periods, but they could potentially become MCUs since they still have an open N26 account.
2. Low Activity - 1%-20%: These MCUs distinctly have a large share of 1-2 MCU periods. It means that they have only tried using their card for a couple of stretches, but mostly are lapsed. This is only an assumption that we will check in later sections.
3. Mid activity - 20%-90%: There probably is a difference between 21% MCU and 89% MCU, but from the chart above we see that all customers in this large gap are lapsing and returning to card usage, because their 3+ mcu period share is around 80%. These customers are either not convinced or are using N26 cards for a specific purpose that does not lead to every-day use.
4. Continuous activity - 90%-100%: These customers are using their card continuously. We can see that 80% of them have 1-2 MCU periods, so they don't lapse often. More likely when they lapse it causes them dropping to lower brackets and this also signifies a change to different types of MCU.

You might assume that the 90%-100% MCU bracket contains mostly new customers, but in general that is not the case. It was [investigated earlier](https://research.tech26.de/reports/20211206_mcu_insights.html#33) that high activity customers have almost equally high numbers of older MCUs. We will not be looking further into the customer tenure here, refer to the linked study for this info.

# 4. MCU categorisation by lapse<a name="section4"></a>
Finally, let's look at how these customer categories differ by their lapse behaviour. We have two types of lapsed MCUs - dormant and lapsed. It was noticed that 90% of customers who return after lapse do it in 5 months. Therefore we can assume that MCU with over 5 months of inactivity is unlikely to return. This outlines two distinct categories of inactive customers:
1. Lapsed - last MCU activity ended >=6 months before the date of this study.
2. Dormant - customer was an MCU <6 months before the date of this study.

We can now look at the MCU activity rates to see whether they show different lapse behaviour. For this purpose we are not including ftMAUs since they have never lapsed.

In [13]:
data = mcu_rate_vs_lapse
xaxis = "activity_level"
yaxis = "customer_percentage"
col_split = "mcu_status"

x_axis_title = "MCU activity level"
y_axis_title = "Percentage of MCUs within activity level"
chart_title = "ftMAUs split by their lapse status"

column_order = [
    "None (0%)",
    "Low (1%-20%)",
    "Sporadic (20%-40%)",
    "Mid (40%-70%)",
    "High (70%-90%)",
    "Continuous (90%-100%)",
]

base = alt.Chart(data).encode(
    x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title), sort=column_order)
)

bar = base.mark_bar(size=100).encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(),
        stack="normalize",
    ),
    color=alt.Color(
        col_split, legend=alt.Legend(title="MCU status", legendX=250, legendY=-55)
    ),
)
bar_inside_labels = base.mark_text(size=20, dx=0, dy=-8).encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(),
        stack="normalize",
    ),
    color=alt.Color(col_split, legend=None),
    text=alt.Text("bracket_rate:Q", format=".0%"),
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O", sort=column_order),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)

bar_select = bar.mark_bar(size=100).encode(
    y=alt.Y(yaxis + ":Q", axis=None, scale=alt.Scale(), stack="normalize"),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)

alt.layer(
    bar + bar_inside_labels,
    bar_select + bar_inside_labels,
    selectors
    # text_bar
).properties(title=chart_title).resolve_scale(y="independent").configure(
    background="#fff"
)

alt.LayerChart(...)

We can again make very clear conclusions about the first and the last bracket. In the _Low (1%-20%)_ bracket we see a lot of lapsed users. In conjunction with the previous chart we know that most users here have 1-2 MCU periods. This suggests a conclusion that the likelihood of these customers returning is very low. Most of this column are customers who tried using the card and then lapsed, but did not close their N26 account.

If we now look at the _Continuous (90%-100%)_ bracket - it is exclusively all active users. A very clear and distinct customer group.

From earlier we know that lapsed MCUs are unlikely to return to using their card. Having this in mind we can label them as a group of customers on their own, despite their previous MCU activity. Note that it's possible that lapsed customers with higher past activity could be more likely to be just dormant, but we will look at this in the future and update might follow. For now we will label them as lapsed.

# 5. A word on MCU periods vs. activity vs. lapse<a name="section5"></a>
Before the categorisation let's discuss how MCU periods relate with overall activity and lapse. Imagine a customer who has been inactive for a couple of months since account opening but since then is using N26 card continuously for the rest of the year. At the moment such customers would likely be the same as all the customers in >90% activity bracket, the only difference is that they had a delayed start.

It would probably be possible to assign these customers to the same category despite a slow start. We can identify these customers by the combination of their MCU traits. a) this customer is currently **Active**; b) he should have **1-2 MCU periods** showing that he is not a frequent lapser and matches to 1-2 periods that 80% of continuous MCUs have. c) His overall **MCU rate should be over 70%**, so that we don't include undecided MCUs.

We could treat these customers the same as continuous MCUs, because based on these conditions they are currently on the way to become continuous MCUs, but based on their passive period it might be difficult to reach 90% to be officially included in this bracket.

This is just a thought for discussion, as currently such categorisation will not be applied. Further look into behaviour of these customers will help to decide whether it makes sense to include them into the continuous user bracket.

# 6. Final categorisation<a name="section6"></a>
Finally we can connect all the information we processed into final groups of customers based on their MCU behaviour.
1. Continuous MCUs - all active MCUs with over 90% of MCU activity. We can expect these customers to be the drivers of all KPIs. An assumption that can later be tested and hopefully confirmed.
2. Sporadic MCUs - all active MCUs that have 3+ MCU periods. Showing that they tend to lapse and return using their card.
3. Undecided MCUs - these are all active MCUs that are <90% with their overall activity and have 1-2 MCU periods. Showing that they don't lapse frequently. These MCUs should either end up with Sporadic MCUs if they stop and return a couple of more times, or end up in the continuous MCU bracket if they remain continuously active.
4. Dormant MCUs - all inactive MCUs that have been inactive for less than 6 months. They can still potentially return using their card and could show similar traits to sporadic MCUs.
5. Lapsed MCUs - all inactive MCUs that have been inactive for over 5 months. They are unlikely to return using their card.
6. Potential MCUs - these are the customers that have topped up and have not used their card yet, but have an open N26 account. They are not really MCUs, but for some analyses it might be useful to include this group.
Let's see how this split looks on our current MCU base.



In [14]:
data = mcu_categories
xaxis = "mcu_category"
yaxis = "customers"

x_axis_title = "MCU category"
y_axis_title = "Percentage of customers"
chart_title = "MCU categories"

column_order = ["Continuous", "Sporadic", "Undecided", "Dormant", "Lapsed", "Potential"]

base = alt.Chart(data).encode(
    x=alt.X(xaxis + ":O", axis=alt.Axis(title=x_axis_title), sort=column_order)
)

bar = base.mark_bar(size=100).encode(
    y=alt.Y(
        yaxis + ":Q",
        axis=alt.Axis(title=y_axis_title, format=".0%", grid=True),
        scale=alt.Scale(),
    ),
    color=alt.Color(xaxis, legend=None),
)
bar_labels = bar.mark_text(dx=40, dy=25, size=40).encode(
    y=alt.Y(yaxis + ":Q", axis=None), text=alt.Text(yaxis + ":N", format=".0%")
)

nearest = alt.selection(
    type="single", nearest=True, on="mouseover", fields=[xaxis], empty="none"
)
selectors = (
    base.mark_point()
    .encode(
        alt.X(xaxis + ":O", sort=column_order),
        opacity=alt.value(0),
    )
    .add_selection(nearest)
)
text_bar = base.mark_text(align="left", dx=-33, dy=25, size=40, color="#fff").encode(
    y=alt.Y(yaxis + ":Q", axis=None),
    text=alt.condition(nearest, yaxis + ":Q", alt.value(" "), format=".0%"),
)
bar_select = bar.mark_bar(size=100).encode(
    y=alt.Y(yaxis + ":Q", axis=None),
    opacity=alt.condition(nearest, alt.value(1), alt.value(0)),
)

alt.layer(bar, bar_labels, selectors, bar_select, text_bar).properties(
    title=chart_title
).resolve_scale(y="independent").configure(background="#fff")

alt.LayerChart(...)

Now that we have these groups of customers, we can go on further studies to see if there are behavioural differences to them. The categorisation is subject to change in the future in case this proves to be inefficient.